## **Preparing cnn file and dataset**

In [ ]:
!pip install -q tensorflow-model-optimization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Loading files
import pickle

keras_file = '/content/drive/My Drive/Colab Notebooks/Capstone/testing/Outdoor_Living/outdoor_living.h5'
X = pickle.load(open("/content/drive/My Drive/Colab Notebooks/Capstone/testing/Outdoor_Living/X_Outdoor_Living.pickle", "rb"))
y = pickle.load(open("/content/drive/My Drive/Colab Notebooks/Capstone/testing/Outdoor_Living/y_Outdoor_Living.pickle", "rb"))
categories = pickle.load(open("/content/drive/My Drive/Colab Notebooks/Capstone/testing/Outdoor_Living/Outdoor_Living_categories.pickle", "rb"))

In [ ]:
# Prepare dataset, we need to fit this to new pruned model.
# Accuracy should be the same or higher

from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)
num_classes = len(categories)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.0
X_test /= 255.0
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(2660, 90, 90, 1)
(2660, 38)
(1140, 90, 90, 1)
(1140, 38)


## **Building prune model**

In [ ]:
# Build and compile new pruned model
import numpy as np
import tensorflow as tf
from tensorflow_model_optimization.sparsity import keras as sparsity

# Load the serialized model
loaded_model = tf.keras.models.load_model(keras_file)

# define parameters
epochs = 10
num_train_samples = X_train.shape[0]
batch_size = 100
end_step = np.ceil(1.0 * num_train_samples / batch_size).astype(np.int32) * epochs
print(end_step)

new_pruning_params = {
      'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.50,
                                                   final_sparsity=0.90,
                                                   begin_step=0,
                                                   end_step=end_step,
                                                   frequency=100)
}

new_pruned_model = sparsity.prune_low_magnitude(loaded_model, **new_pruning_params)
new_pruned_model.summary()

new_pruned_model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py:2191: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_conv2d_1 (None, 90, 90, 32)        610       
_________________________________________________________________
prune_low_magnitude_max_pool (None, 45, 45, 32)        1         
_________________________________________________________________
prune_low_magnitude_dropout_ (None, 45, 45, 32)        1         
_________________________________________________________________
prune_low_magnitude_conv2d_1 (None, 45, 45, 64)        36930     
_________________________________________________________________
prune_low_magnitude_max_pool (None, 22, 22, 64)        1         
_________________________________________________________________
prune_low_magnitude_dropout_ (None, 22, 22, 64)        1         
_________________________________________________________________
prune_low_magnitude_flatten_ (None, 30976)            

In [ ]:
logdir = '/content/drive/My Drive/Colab Notebooks/Capstone/testing/pruning_log'

callbacks = [
    sparsity.UpdatePruningStep(),
    sparsity.PruningSummaries(log_dir=logdir, profile_batch=0)
]

new_pruned_model.fit(X_train, y_train,
          epochs=epochs,
          verbose=1,
          callbacks=callbacks,
          validation_data=(X_test, y_test))

score = new_pruned_model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
84/84 [==============================] - 36s 30ms/step - loss: 0.3821 - accuracy: 0.9226 - val_loss: 1.5311 - val_accuracy: 0.7132
Epoch 2/10
84/84 [==============================] - 2s 28ms/step - loss: 0.3728 - accuracy: 0.9113 - val_loss: 1.5563 - val_accuracy: 0.6982
Epoch 3/10
84/84 [==============================] - 2s 28ms/step - loss: 0.2783 - accuracy: 0.9335 - val_loss: 1.6210 - val_accuracy: 0.7000
Epoch 4/10
84/84 [==============================] - 2s 28ms/step - loss: 0.2453 - accuracy: 0.9383 - val_loss: 1.5955 - val_accuracy: 0.7096
Epoch 5/10
84/84 [==============================] - 2s 28ms/step - loss: 0.2129 - accuracy: 0.9447 - val_loss: 1.6253 - val_accuracy: 0.7114
Epoch 6/10
84/84 [==============================] - 2s 29ms/step - loss: 0.2030 - accuracy: 0.9526 - val_loss: 1.5338 - val_accuracy: 0.7053
Epoch 7/10
84/8

In [ ]:
final_model = sparsity.strip_pruning(new_pruned_model)
final_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 90, 90, 32)        320       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 45, 45, 32)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 45, 45, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 45, 45, 64)        18496     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 22, 22, 64)        0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 22, 22, 64)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 30976)            

## **Saving pruned model**

In [ ]:
new_pruned_keras_file = '/content/drive/My Drive/Colab Notebooks/Capstone/testing/Outdoor_Living/outdoor_living_pruned.h5'
print("Saving pruned model to: ", new_pruned_keras_file)
tf.keras.models.save_model(final_model, new_pruned_keras_file, include_optimizer=False)

Saving pruned model to:  /content/drive/My Drive/Colab Notebooks/Capstone/testing/Outdoor_Living/outdoor_living_pruned_256.h5


## **Reducing pruned model to x5 through zip3**

In [ ]:
import tempfile
import zipfile
import os

# Zip the .h5 model file
zip3 = '/content/drive/My Drive/Colab Notebooks/Capstone/testing/Outdoor_Living/outdoor_living_pruned.h5.zip3'
with zipfile.ZipFile(zip3, "w", compression=zipfile.ZIP_DEFLATED) as f:
    f.write(new_pruned_keras_file)
print(
    "Size of the pruned model before compression: %.2f Mb"
    % (os.path.getsize(new_pruned_keras_file) / float(2 ** 20))
)
print(
    "Size of the pruned model after compression: %.2f Mb"
    % (os.path.getsize(zip3) / float(2 ** 20))
)

Size of the pruned model before compression: 121.25 Mb
Size of the pruned model after compression: 51.70 Mb
